In [ ]:
import torch
import pickle
import random
import torchvision
import numpy as np
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.training.supervised import GSS_greedy
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy

In [ ]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False
cudnn.deterministic = True

In [ ]:
train_set = CIFAR10('../Dataset/CIFAR10', train=True, download=True)
test_set = CIFAR10('../Dataset/CIFAR10', train=False, download=True)

transforms_group = dict(
       train=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       ),
       eval=(
       transforms.Compose(
              [
              transforms.ToTensor(),
              ]
       ),
       None,
       )
)

train_set = AvalancheDataset(train_set, transform_groups=transforms_group, initial_transform_group="train")
test_set = AvalancheDataset(test_set, transform_groups=transforms_group, initial_transform_group="eval")

In [ ]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

num_class = 10
incremental = 5

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False
                        )

model = torchvision.models.resnet18(pretrained=False, num_classes=num_class)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=1e-2)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_LwF_cifar100")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceTime(),
    loggers=[interactive_logger, tensor_logger])

## GSS-Greedy

In [ ]:
train_batch = 10
eval_batch = 10
epoch = 10

strategies = GSS_greedy(model, optimizer, criterion, mem_size=200, mem_strength=10, train_epochs=epoch, device=device, train_mb_size=train_batch, eval_mb_size=eval_batch, evaluator=eval_plugin)

In [ ]:
print("Starting experiment...")
results = []

for experience in scenario.train_stream:
    print("Start training on experience ", experience.current_experience)
    strategies.train(experience)
    print("End training on experience ", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(strategies.eval(scenario.test_stream[:]))

# save result
log_path = "./logs.txt"
with open(log_path, 'wb') as lf:
    pickle.dump(results, lf)